In [1]:
from music21 import *
from comp2 import *

In [2]:
melody_in: stream.Stream = parse_midi("example_inputs/eyeofthetiger.mid")
melody_in = melody_in.transpose(4)

print("Original melody:")
melody_in.show('midi');

Original melody:


In [3]:
print("Simple triads:")
simple_triads = get_triads(melody_in)

Simple triads:
['iii', 'ii', 'ii', 'ii', 'iii', 'iii', 'ii', 'iii', 'iii', 'ii', 'ii', 'ii', 'iii', 'iii', 'ii', 'iii', 'IV', 'I', 'IV', 'iii', 'IV', 'I', 'IV', 'vi', 'ii']


In [4]:
out_score = stream.Score()
out_score.insert(0, melody_in)
out_score.insert(0, simple_triads)

out_score.show('midi')

In [5]:
markov_table = generate_markov_table()
substituted_chords = substitute_chords(simple_triads, markov_table)
for i in range(5):
    substituted_chords = substitute_chords(substituted_chords, markov_table)

out_score = stream.Score()
out_score.insert(0, melody_in)
out_score.insert(0, substituted_chords)

out_score.show('midi')

Total number of states: 2744

new chords:
['iii', 'iisus4', 'ii2', 'ii2', 'iii2', 'iii', 'vi2', 'iii2', 'iii2', 'ii2', 'ii2', 'ii2', 'iiisus4', 'iii', 'ii2', 'iii2', 'IV2', 'I2', 'IV7', 'i2', 'iv6', 'I2', 'IV2', 'VI2', 'ii']

new chords:
['iii2', 'v2', 'ii', 'ii2', 'iii2', 'iii2', 'vi2', 'IV', 'iii76', 'vi2', 'ii7', 'v7', 'vi2', 'iii7', 'ii7', 'iii7', 'IV2', 'I7', 'IV6', 'ii7', 'iv7', 'vii6', 'IV2', 'VII7', 'ii']

new chords:
['iii2', 'v2', 'ii7', 'ii', 'iii2', 'vi2', 'vi7', 'IV2', 'I', 'vi', 'ii2', 'v2', 'i7', 'ii2', 'ii2', 'iii7', 'IV7', 'I2', 'IV', 'VII2', 'iv', 'vii2', 'iv', 'iv7', 'ii2']

new chords:
['iii2', 'vsus4', 'ii7', 'ii7', 'iii2', 'vi2', 'ii2', 'IV76', 'ii2', 'vi7', 'ii7', 'II7', 'v2', 'i', 'VII2', 'iii76', 'vi', 'I7', 'IV2', 'VII2', 'vii', 'vii', 'iii2', 'iv2', 'ii2']

new chords:
['iii76', 'i7', 'ii', 'ii', 'iiisus4', 'vi7', 'iv2', 'III2', 'ii76', 'v', 'ii2', 'iii2', 'v', 'vi2', 'V2', 'vii7', 'vi', 'I', 'IV2', 'VII2', 'vii6', 'vii2', 'iii2', 'vi2', 'ii76']

new chords:


In [6]:
import torch
from preprocess import *
from bigram import BigramModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_save_path = '/mnt/c/Users/jwest/Desktop/algocomps/comp2-markov/parkergpt_trained.pt'
checkpoint = torch.load(model_save_path, map_location=device)

model = BigramModel(
    device=device,
    block_size=checkpoint['block_size'],
    vocab_size=checkpoint['vocab_size'],
    n_embed=checkpoint['n_embed']
)

# Load the saved weights
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [7]:
gen_tokens = model.generate(
    idx=torch.zeros((1, 1), dtype=torch.long).to(device),
    max_length=100  # Generate sequence
)[0].tolist()

# Convert tokens to MIDI
gen_stream = sequence_to_score(tokens_to_sequence(gen_tokens))
gen_stream.show('midi')

In [ ]:
def splice_melody(melody: stream.Stream) -> stream.Stream:
    out = []
    pos = 0  # position in melody (in 24ths of a quarter note)
    cur_segment_length = 0
    gen_tokens = None
    take = 'melody'
    gen_index = 0  # Track position in generated content
    
    while pos < melody.duration.quarterLength * 24:
        if take == 'melody':
            # take next element from melody
            next_elem = melody.flatten().getElementAtOrBefore(pos / 24)
            if next_elem is None:
                break
            out.append(next_elem)
            pos += int(next_elem.quarterLength * 24)
            cur_segment_length += int(next_elem.quarterLength * 24)
            
            if cur_segment_length > 144:
                take = 'model'
                # generate 100 tokens with current melody as context
                context = out[-6:]
                context = make_tokens(score_to_sequence(context))
                print(context)
                gen_tokens = sequence_to_score(tokens_to_sequence(model.generate(
                    idx=torch.tensor(context, dtype=torch.long).unsqueeze(0).to(device),
                    max_length=100
                )[0].tolist()))
                cur_segment_length = 0
                gen_index = 0  # Reset the index counter
        else:
            # Get elements by index instead of by offset
            flat_gen = gen_tokens.flatten().notes
            if gen_index >= len(flat_gen):
                take = 'melody'
                cur_segment_length = 0
                continue
                
            next_elem = flat_gen[gen_index]
            gen_index += 1  # Increment index
            
            # Create a COPY of the element to prevent ID conflicts
            if isinstance(next_elem, note.Note):
                next_elem = note.Note(pitch=next_elem.pitch, quarterLength=next_elem.quarterLength)
            elif isinstance(next_elem, note.Rest):
                next_elem = note.Rest(quarterLength=next_elem.quarterLength)
            
            next_elem.offset = pos / 24
            
            out.append(next_elem)
            pos += int(next_elem.quarterLength * 24)
            cur_segment_length += int(next_elem.quarterLength * 24)
            
            if cur_segment_length > 144:
                take = 'melody'
                cur_segment_length = 0
    
    return stream.Stream(out)


In [15]:
res_melody = splice_melody(melody_in)

out_score = stream.Score()
out_score.insert(0, res_melody)
out_score.insert(0, substituted_chords)

out_score.show('midi')

[1787, 1737, 1837, 1793]
[1837, 1962, 31, 1837]
[1731, 1831, 1849, 49, 49, 49]
[1787, 1781, 31, 1749]
[1793, 31, 1787, 1737]
[1787, 1837, 1787, 37]
[31, 1837, 1787, 1737]
[1781, 1781, 31, 1737]


In [16]:
out_score.write('midi', 'tiger_out.mid')

'tiger_out.mid'